# Immobilienpreis-Rechner

Eine grafische Anwendung (GUI) zur schnellen Schätzung des Immobilienwertes in Deutschland.

## Eingabeparameter

| Eingabe | Typ |
|---|---|
| Grundstücksfläche (m²) | Zahleneingabe |
| Wohnfläche (m²) | Zahleneingabe |
| Baujahr | Zahleneingabe |
| Bundesland | Dropdown |
| Stadt oder Land | Dropdown |
| Hausart | Dropdown |
| Ausstattung | Dropdown |
| Architektenhaus | Dropdown |
| Makler | Dropdown |
| Denkmalschutz | Dropdown |

## Berechnungsformel

```
Basispreis = Grundstück × 160 €/m² + Wohnfläche × 2.500 €/m² − Altersabschlag
Gesamtpreis = Basispreis × Faktor(Bundesland) × Faktor(Ort) × ... (aus datei.json)
```

## Voraussetzungen

Python 3.x — nur Standardbibliotheken: `tkinter`, `json`, `datetime`

Die Datei `datei.json` muss sich im selben Ordner befinden.

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import datetime
import json

# Preiskonstanten – hier zentral definiert, damit man sie leicht anpassen kann
PREIS_GRUNDSTUECK_PRO_QM = 160
PREIS_WOHNFLAECHE_PRO_QM = 2500
ALTERSABSCHLAG_PRO_JAHR = 0.1

# Daten aus der JSON-Datei laden
with open("datei.json", "r", encoding="utf-8") as file:
    datei = json.load(file)

# Hauptfenster erstellen
fenster = tk.Tk()
fenster.title("Immobilienpreis-Rechner")
fenster.geometry('650x570')
fenster.configure(bg="#A9BA9D")
fenster.option_add("*Font", "Arial 11")

# Titeltext im Fenster anzeigen
label_titel = tk.Label(
    fenster,
    text="Beantworten Sie bitte 10 einfache Fragen,\num eine sofortige Schätzung des Immobilienwertes zu erhalten:",
    wraplength=500,
    justify='center',
    font=("Arial", 14, "bold"),
    bg="#A9BA9D"
)
label_titel.grid(row=0, column=0, columnspan=2, pady=10)

# Texteingabefelder für Grundstücksfläche, Wohnfläche und Baujahr
tk.Label(fenster, text="Fläche des Grundstücks (m²):", bg="#A9BA9D").grid(row=3, column=0, sticky='w', padx=15, pady=5)
entry_grundstueck = tk.Entry(fenster, width=22)
entry_grundstueck.grid(row=3, column=1, padx=15, pady=5)

tk.Label(fenster, text="Wohnfläche (m²):", bg="#A9BA9D").grid(row=4, column=0, sticky='w', padx=15, pady=5)
entry_wohnflaeche = tk.Entry(fenster, width=22)
entry_wohnflaeche.grid(row=4, column=1, padx=15, pady=5)

tk.Label(fenster, text="Baujahr:", bg="#A9BA9D").grid(row=5, column=0, sticky='w', padx=15, pady=5)
entry_baujahr = tk.Entry(fenster, width=22)
entry_baujahr.grid(row=5, column=1, padx=15, pady=5)

# Mit Enter-Taste zum nächsten Feld springen
entry_grundstueck.bind("<Return>", lambda e: entry_wohnflaeche.focus())
entry_wohnflaeche.bind("<Return>", lambda e: entry_baujahr.focus())

# Dropdown-Menüs definieren: (Beschriftung, JSON-Schlüssel, Zeilenposition)
dropdown_config = [
    ("Bundesland:", "bundesland", 6),
    ("Stadt oder Land?", "ort", 7),
    ("Hausart:", "hausart", 8),
    ("Ausstattung:", "ausstattung", 9),
    ("Architektenhaus?", "architekt", 10),
    ("Makler?", "makler", 11),
    ("Denkmalschutz?", "denkmalschutz", 12),
]

# Stil für Dropdown-Menüs festlegen
style = ttk.Style()
style.theme_use("default")
style.configure("Custom.TCombobox",
                fieldbackground="#A9BA9D",
                background="#A9BA9D",
                foreground="black")

# Dropdown-Menüs erstellen und in einem Wörterbuch speichern
auswahlen = {}
for text, schluessel, zeile in dropdown_config:
    tk.Label(fenster, text=text, bg="#A9BA9D").grid(row=zeile, column=0, sticky='w', padx=15)
    kombobox = ttk.Combobox(fenster, values=list(datei[schluessel].keys()),
                            state="readonly", style="Custom.TCombobox")
    kombobox.grid(row=zeile, column=1, padx=15, pady=5)
    auswahlen[schluessel] = kombobox

# Ergebnisanzeige
label_ergebnis = tk.Label(fenster, text="", font=('Arial', 14, 'bold'), fg="black", bg="#A9BA9D")
label_ergebnis.grid(row=23, column=0, columnspan=2, pady=10, padx=15)


def berechne():
    """Liest alle Eingaben, prüft sie und berechnet den geschätzten Immobilienpreis."""

    aktuelles_jahr = datetime.datetime.now().year
    fehler = []

    # Texteingaben lesen und Komma durch Punkt ersetzen (z.B. 1500,5 -> 1500.5)
    grundstueck_text = entry_grundstueck.get().replace(",", ".")
    wohnflaeche_text = entry_wohnflaeche.get().replace(",", ".")
    baujahr_text = entry_baujahr.get()

    # Zahlenwerte prüfen und konvertieren
    try:
        grundstueck = float(grundstueck_text)
        if grundstueck <= 0:
            fehler.append("Grundstücksfläche muss größer als 0 sein.")
    except ValueError:
        fehler.append("Bitte eine gültige Zahl für die Grundstücksfläche eingeben (z.B. 1500 oder 1500.5).")
        grundstueck = None

    try:
        wohnflaeche = float(wohnflaeche_text)
        if wohnflaeche <= 0:
            fehler.append("Wohnfläche muss größer als 0 sein.")
    except ValueError:
        fehler.append("Bitte eine gültige Zahl für die Wohnfläche eingeben.")
        wohnflaeche = None

    try:
        baujahr = int(baujahr_text)
        if baujahr <= 1000 or baujahr > aktuelles_jahr:
            fehler.append(f"Baujahr muss zwischen 1000 und {aktuelles_jahr} liegen.")
    except ValueError:
        fehler.append("Bitte ein gültiges Baujahr eingeben.")
        baujahr = None

    # Alle Dropdown-Menüs prüfen
    for text, schluessel, _ in dropdown_config:
        if not auswahlen[schluessel].get():
            fehler.append(f"Bitte '{text}' auswählen.")

    # Wenn es Fehler gibt, alle auf einmal anzeigen und abbrechen
    if fehler:
        messagebox.showerror("Fehler", "\n".join(fehler))
        return

    # Basispreis berechnen: Grundstück + Wohnfläche − Altersabschlag
    grundstueck_preis = grundstueck * PREIS_GRUNDSTUECK_PRO_QM
    haus_preis = wohnflaeche * PREIS_WOHNFLAECHE_PRO_QM
    altersabschlag = max(0, (aktuelles_jahr - baujahr) * ALTERSABSCHLAG_PRO_JAHR)
    gesamtpreis = grundstueck_preis + haus_preis - altersabschlag

    # Faktoren aus den Dropdown-Menüs anwenden (z.B. Hamburg × 2.5)
    for _, schluessel, _ in dropdown_config:
        auswahl = auswahlen[schluessel].get()
        gesamtpreis *= datei[schluessel].get(auswahl, 1)

    # Ergebnis anzeigen
    label_ergebnis.config(text=f"Geschätzter Preis: {int(gesamtpreis):,} €".replace(",", "."))


def zuruecksetzen():
    """Setzt alle Felder auf den Ausgangszustand zurück."""
    entry_grundstueck.delete(0, 'end')
    entry_wohnflaeche.delete(0, 'end')
    entry_baujahr.delete(0, 'end')
    for kombobox in auswahlen.values():
        kombobox.set('')
    label_ergebnis.config(text="")


# Buttons erstellen
tk.Button(fenster, text="Berechnen", command=berechne).grid(row=22, column=0, pady=20)
tk.Button(fenster, text="Zurücksetzen", command=zuruecksetzen).grid(row=22, column=1, pady=20)

# Fenster öffnen und auf Eingaben warten
fenster.mainloop()
